## 🧠 Non-Inferiority Test: AI vs. Human (Weighted F1 Bootstrap)

This notebook computes a non-inferiority test for multiple AI classifiers (AI1-AI4) against a single Human classifier (H1) using a **paired, stratified bootstrap** on the **Weighted F1-score (W-F1)** difference.

The F1-score is weighted by `log(1 + concept_count)`.

### 1. Configuration
Set the paths and parameters for the test.

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import re
from tqdm.notebook import tqdm

# ----------------------------------------------------------------
# ⚙️ CONFIGURATION
# ----------------------------------------------------------------

# --- Statistical Parameters ---
# ⚠️ Define your non-inferiority margin (Δ).
NON_INFERIORITY_MARGIN = 0.05
BOOTSTRAP_ITERATIONS = 10000
ALPHA = 0.05  # For a 95% one-sided CI (or 90% two-sided CI)

# --- Classifier IDs ---
HUMAN_ARM_ID = "H1"
AI_ARM_IDS = ["AI1", "AI2", "AI3", "AI4"] # Test all AI arms

# --- Path Configuration ---
try:
    SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    SCRIPT_DIR = os.getcwd()  # Fallback for interactive notebooks

GOLD_DIR = os.path.join(SCRIPT_DIR, "results", "phaseA_gold_final")
CONCEPTSET_DIR = os.path.abspath(os.path.join(SCRIPT_DIR, "..", "ConceptSets"))
RECORD_COUNT_PATH = os.path.join(SCRIPT_DIR, "ConceptRecordCounts.csv")

print(f"Notebook directory: {SCRIPT_DIR}")
print(f"Gold Standard directory: {GOLD_DIR}")
print(f"ConceptSet directory: {CONCEPTSET_DIR}")
print(f"Record Count file: {RECORD_COUNT_PATH}")
print(f"---")
print(f"Comparing AI arms: {', '.join(AI_ARM_IDS)}")
print(f"Against Human arm: {HUMAN_ARM_ID}")
print(f"Non-Inferiority Margin (Δ): {NON_INFERIORITY_MARGIN}")
print(f"Test: W-F1(AI) - W-F1(Human) > -{NON_INFERIORITY_MARGIN}")

### 2. Helper Functions
These functions find and load the concept files for each task and classifier.

In [ ]:
def find_concept_file(disease_prefix, arm_id, concept_dir):
    """Finds the includedConcepts.csv file for a specific disease prefix and arm ID."""
    pattern = re.compile(rf"\[{disease_prefix}\](.*?)\[{arm_id}\]", re.IGNORECASE)
    online_pattern = re.compile(r"ONLINE", re.IGNORECASE)
    
    for folder_name in os.listdir(concept_dir):
        folder_path = os.path.join(concept_dir, folder_name)
        if not os.path.isdir(folder_path):
            continue
            
        if pattern.search(folder_name) and not online_pattern.search(folder_name):
            csv_path = os.path.join(folder_path, "includedConcepts.csv")
            if os.path.exists(csv_path):
                return csv_path
    return None

def load_concepts(file_path):
    """Loads concept IDs from a CSV file into a set."""
    if file_path is None or not os.path.exists(file_path):
        print(f"⚠️ File not found, returning empty set: {file_path}")
        return set()
    try:
        df = pd.read_csv(file_path, dtype=str)
        df.columns = [c.strip() for c in df.columns]
        
        target_col = None
        for col in df.columns:
            if col.lower() == "conceptid":
                target_col = col
                break
        
        if target_col is None:
             for col in df.columns:
                 if "concept" in col.lower() and "id" in col.lower():
                    target_col = col
                    break
        
        if target_col is None:
            print(f"⚠️ Could not find 'conceptId' in {file_path}. Using first column.")
            target_col = df.columns[0]
            
        vals = df[target_col].dropna().astype(str).str.strip().str.replace(r"\.0+$", "", regex=True)
        return set(vals[vals != ""])
    except Exception as e:
        print(f"❌ Error loading {file_path}: {e}")
        return set()


### 3. Load Concept Weights
Load the `ConceptRecordCounts.csv` file and create the `log(1 + count)` weight for each concept.

In [ ]:
if not os.path.exists(RECORD_COUNT_PATH):
    print(f"❌ CRITICAL ERROR: ConceptRecordCounts.csv not found at {RECORD_COUNT_PATH}")
    print("Please ensure the file is in the correct location.")
    df_counts = pd.DataFrame(columns=["concept_id", "record_count"])
else:
    df_counts = pd.read_csv(RECORD_COUNT_PATH, dtype=str)
    print(f"Loaded {len(df_counts)} records from {RECORD_COUNT_PATH}")

# Ensure correct dtypes
df_counts['concept_id'] = df_counts['concept_id'].astype(str).str.strip().str.replace(r"\.0+$", "", regex=True)
df_counts['record_count'] = pd.to_numeric(df_counts['record_count'], errors='coerce').fillna(0)

# Calculate the log(1+count) weight
df_counts['weight'] = np.log1p(df_counts['record_count'])

# Create a lookup dictionary (Series) for fast mapping
weight_lookup = df_counts.set_index('concept_id')['weight']

print(f"Created weight lookup for {len(weight_lookup)} concepts.")
display(weight_lookup.head())

### 4. Build Master Item-Level Dataset

We now loop through all tasks (diseases) and build one large DataFrame. Each row represents a single concept (item) and its classification by all parties, plus its concept weight.

In [ ]:
all_items = []
all_tasks_found = []
classifier_columns = ['human'] + [arm.lower() for arm in AI_ARM_IDS]

gold_files = glob.glob(os.path.join(GOLD_DIR, "*_Gold_Standard_FINAL.csv"))
print(f"Found {len(gold_files)} Gold Standard files.")

for gs_file in gold_files:
    base_name = os.path.basename(gs_file)
    match = re.search(r"^(C\d+)", base_name, re.IGNORECASE)
    if not match:
        print(f"Skipping {base_name}: Could not parse disease prefix.")
        continue
        
    disease_prefix = match.group(1)
    task_name = base_name.replace("_Gold_Standard_FINAL.csv", "")
    
    # Find corresponding classifier files
    human_file = find_concept_file(disease_prefix, HUMAN_ARM_ID, CONCEPTSET_DIR)
    ai_files = {arm: find_concept_file(disease_prefix, arm, CONCEPTSET_DIR) for arm in AI_ARM_IDS}
    
    all_files_found = human_file is not None and all(ai_files.values())
    if not all_files_found:
        print(f"\n⚠️ Skipping {task_name} (prefix {disease_prefix}):")
        if not human_file: 
            print(f"  - Missing file for {HUMAN_ARM_ID}")
        for arm, f in ai_files.items():
            if f is None:
                print(f"  - Missing file for {arm}")
        continue
        
    # Load concept sets
    gs_concepts = load_concepts(gs_file)
    human_concepts = load_concepts(human_file)
    ai_concepts = {arm: load_concepts(f) for arm, f in ai_files.items()}
    
    # Define the 'universe' of items for this task
    universe = gs_concepts | human_concepts
    for arm_concepts in ai_concepts.values():
        universe.update(arm_concepts)
    
    if not universe:
        print(f"Skipping {task_name}: No concepts found.")
        continue

    # Create item-level records
    for concept_id in universe:
        record = {
            'task_id': task_name,
            'concept_id': concept_id,
            'gold': concept_id in gs_concepts,
            'human': concept_id in human_concepts,
            'weight': weight_lookup.get(concept_id, 0.0) # Get weight, default to 0.0 (log(1+0))
        }
        # Add AI classifications
        for arm in AI_ARM_IDS:
            record[arm.lower()] = concept_id in ai_concepts[arm]
        
        all_items.append(record)
    
    print(f"Processed {task_name}: {len(universe)} items.")
    all_tasks_found.append(task_name)

# Create the final DataFrame
df_items = pd.DataFrame(all_items)

print("\n" + "="*30)
print(f"Master dataset created.")
print(f"Total tasks included: {len(all_tasks_found)}")
print(f"Total items (concepts): {len(df_items)}")

# ----------------------------------------------------------------
# ⚠️ BUG FIX: Check if df_items is empty before proceeding
# ----------------------------------------------------------------
if df_items.empty:
    print("\n❌ CRITICAL ERROR: No data was loaded into the master DataFrame.")
    print("This happens if no tasks were processed (e.g., missing Gold Standard or arm files).")
    print("Please check the log output above. Halting execution.")
    # This will cause a NameError if you try to run subsequent cells, which is intended.
    raise SystemExit("Stopping notebook due to empty master DataFrame.")
else:
    display(df_items.head())


### 5. Define Weighted F1-Score Calculation

This function takes a DataFrame and computes the aggregate **Weighted F1-score** for Human and all AI arms.

In [ ]:
def calculate_weighted_f1_scores(df, classifier_cols):
    """Calculates aggregate Weighted F1 scores for all specified classifiers."""
    
    scores = {}
    # Get the gold and weight columns ONCE, as they are constant for all classifiers
    is_gold = df['gold']
    weights = df['weight']
    
    for clf in classifier_cols:
        # Create boolean mask for this classifier
        is_clf = df[clf]
        
        # Multiply the boolean mask by the weight, then sum
        wtp = (is_gold & is_clf) * weights
        wfp = (~is_gold & is_clf) * weights
        wfn = (is_gold & ~is_clf) * weights
        
        # Sum the weights
        WTP = wtp.sum()
        WFP = wfp.sum()
        WFN = wfn.sum()
        
        denominator = (2 * WTP + WFP + WFN)
        f1 = (2 * WTP) / denominator if denominator > 0 else 0
        scores[clf] = f1
        
    return scores

### 6. Run Stratified Bootstrap

This is the core of the test. We loop 10,000 times. In each loop, we:
1.  Create a new dataset by sampling *with replacement* from each task.
2.  Calculate the aggregate **Weighted F1 scores** for this new dataset.
3.  Store the difference for each AI: $W-F1_{AI_n} - W-F1_{Human}$.

In [ ]:
# Get the original, observed Weighted F1 scores and differences
obs_scores = calculate_weighted_f1_scores(df_items, classifier_columns)
obs_diffs = {}

print("--- Observed Weighted F1 Scores ---")
print(f"Observed W-F1 ({HUMAN_ARM_ID}): {obs_scores['human']:.4f}")
for arm in AI_ARM_IDS:
    arm_key = arm.lower()
    obs_diff = obs_scores[arm_key] - obs_scores['human']
    obs_diffs[arm] = obs_diff
    print(f"Observed W-F1 ({arm}):     {obs_scores[arm_key]:.4f} (Diff: {obs_diff:+.4f})")

# Initialize storage for bootstrap differences
bootstrap_diffs = {arm: [] for arm in AI_ARM_IDS}

tasks = df_items['task_id'].unique()
task_indices = {task: df_items[df_items['task_id'] == task].index for task in tasks}

print(f"\nRunning {BOOTSTRAP_ITERATIONS} bootstrap iterations...")

# Set a fixed random seed for reproducible results
np.random.seed(42)

for _ in tqdm(range(BOOTSTRAP_ITERATIONS)):
    # Create stratified sample
    resampled_indices = []
    for task in tasks:
        indices = task_indices[task]
        # Sample with replacement from this task's items
        sample = np.random.choice(indices, size=len(indices), replace=True)
        resampled_indices.append(sample)
    
    # Combine indices and create the bootstrap dataset
    df_boot = df_items.loc[np.concatenate(resampled_indices)]
    
    # Calculate Weighted F1 scores for this sample
    scores = calculate_weighted_f1_scores(df_boot, classifier_columns)
    f1_h = scores['human']
    
    # Store the difference for each AI arm
    for arm in AI_ARM_IDS:
        f1_ai = scores[arm.lower()]
        bootstrap_diffs[arm].append(f1_ai - f1_h)

print("Bootstrap complete.")

### 7. Results and Non-Inferiority Conclusion

We now analyze the 10,000 Weighted F1 differences for *each AI arm* to build their confidence intervals and make a conclusion.

In [ ]:
print("="*50)
print("    FINAL NON-INFERIORITY TEST RESULTS (WEIGHTED F1)")
print("="*50)

for arm in AI_ARM_IDS:
    arm_key = arm.lower()
    diffs_arr = np.array(bootstrap_diffs[arm])
    
    # For a one-sided test at ALPHA=0.05, we need the 5th percentile.
    # This corresponds to the lower bound of a 90% two-sided CI.
    ci_lower = np.percentile(diffs_arr, 100 * ALPHA)
    ci_upper = np.percentile(diffs_arr, 100 * (1 - ALPHA))
    mean_diff = np.mean(diffs_arr)
    
    # --- The Non-Inferiority Test ---
    # We REJECT the null (H0: AI is inferior) if the lower bound 
    # of our CI is GREATER than the negative margin.
    is_non_inferior = ci_lower > -NON_INFERIORITY_MARGIN
    
    # --- Final Report for this Arm ---
    print(f"\n--- [ {arm} vs. {HUMAN_ARM_ID} ] ---")
    print(f"Non-Inferiority Margin (Δ): {NON_INFERIORITY_MARGIN:.4f}")
    print(f"Hypothesis (H1):      W-F1({arm}) - W-F1({HUMAN_ARM_ID}) > -{NON_INFERIORITY_MARGIN}")
    print("-"*40)
    print(f"Observed W-F1 ({HUMAN_ARM_ID}):  {obs_scores['human']:.4f}")
    print(f"Observed W-F1 ({arm}):     {obs_scores[arm_key]:.4f}")
    print(f"Observed Difference:  {obs_diffs[arm]:+.4f}")
    print(f"Mean Bootstrapped Diff: {mean_diff:+.4f}")
    print(f"90% Confidence Interval: [{ci_lower:+.4f}, {ci_upper:+.4f}]")
    print("-"*40)
    print(f"Test Condition:       Lower Bound > -Margin")
    print(f"Result:               {ci_lower:+.4f} > {-NON_INFERIORITY_MARGIN}")
    print("\n   CONCLUSION:")
    if is_non_inferior:
        print(f"    ✅ The {arm} classifier IS non-inferior to the {HUMAN_ARM_ID} classifier.")
        print("       (The lower bound of the 90% CI is above the non-inferiority margin.)")
    else:
        print(f"    ❌ The {arm} classifier is NOT non-inferior to the {HUMAN_ARM_ID} classifier.")
        print("       (The lower bound of the 90% CI is at or below the non-inferiority margin.)")
    print("-"*40)

print("\n" + "="*50)
print("    All tests complete.")
print("="*50)
